In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('clean.csv')

In [3]:
test = pd.read_csv('clean_test.csv')

In [4]:
train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
train['train']=1
test['train']=0

In [6]:
combined = pd.concat([train,test])

/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
combined['status_group'], labels = pd.factorize(combined['status_group'])

In [8]:
train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

test_df_id = test_df.pop('id')
test_df_statgrp = test_df.pop('status_group')

# train = train_df
train = train_df.sample(frac=0.8)
validate = train_df.sample(frac=0.1)
test = train_df.sample(frac=0.1)


train_target = train.pop('status_group')
validate_target = validate.pop('status_group')

[xsize, ysize] =train.shape
test_target = test.pop('status_group')

/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD, Adam, RMSprop

num_labels = len(np.unique(train_target))
train_target_cat = to_categorical(train_target)
validate_target_cat = to_categorical(validate_target)
test_target_cat = to_categorical(test_target)

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 400:
        lr *= 0.5e-3
    elif epoch > 300:
        lr *= 1e-3
    elif epoch > 200:
        lr *= 1e-2
    elif epoch > 100:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

batch_size = 64
dropout = 0.3
input_size = (1 * ysize * 1)

model = Sequential()
model.add(Dense(256, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))


Using TensorFlow backend.
W0319 22:14:32.534492 140366366037824 deprecation.py:506] From /home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.summary()

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

optim = Adam(lr=lr_schedule(0))

model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

model.fit(train, train_target_cat,  validation_data=(validate, validate_target_cat), epochs=500, batch_size=batch_size, callbacks = callbacks)
# model.fit(train, train_target_cat, epochs=500, batch_size=batch_size, callbacks = callbacks)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               5120      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
__________

Epoch 38/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 39us/step - loss: 0.6627 - acc: 0.7168 - val_loss: 0.6087 - val_acc: 0.7480
Epoch 39/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6588 - acc: 0.7172 - val_loss: 0.6143 - val_acc: 0.7396
Epoch 40/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 35us/step - loss: 0.6600 - acc: 0.7143 - val_loss: 0.6097 - val_acc: 0.7429
Epoch 41/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6584 - acc: 0.7193 - val_loss: 0.6144 - val_acc: 0.7439
Epoch 42/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 35us/step - loss: 0.6558 - acc: 0.7178 - val_loss: 0.6124 - val_acc: 0.7428
Epoch 43/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6537 - acc: 0.7189 - val_loss: 0.6032 - val_acc: 0.7492
Epoch 44/500
Learning rate:  0.001

Epoch 89/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6243 - acc: 0.7358 - val_loss: 0.5749 - val_acc: 0.7604
Epoch 90/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6297 - acc: 0.7336 - val_loss: 0.5713 - val_acc: 0.7604
Epoch 91/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6253 - acc: 0.7339 - val_loss: 0.5713 - val_acc: 0.7567
Epoch 92/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 37us/step - loss: 0.6276 - acc: 0.7340 - val_loss: 0.5704 - val_acc: 0.7554
Epoch 93/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 38us/step - loss: 0.6261 - acc: 0.7343 - val_loss: 0.5570 - val_acc: 0.7652
Epoch 94/500
Learning rate:  0.001
47520/47520 [==============================] - 2s 36us/step - loss: 0.6212 - acc: 0.7387 - val_loss: 0.5721 - val_acc: 0.7635
Epoch 95/500
Learning rate:  0.001

47520/47520 [==============================] - 2s 36us/step - loss: 0.5675 - acc: 0.7624 - val_loss: 0.5230 - val_acc: 0.7835
Epoch 140/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 35us/step - loss: 0.5678 - acc: 0.7612 - val_loss: 0.5235 - val_acc: 0.7813
Epoch 141/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 37us/step - loss: 0.5672 - acc: 0.7615 - val_loss: 0.5231 - val_acc: 0.7813
Epoch 142/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 36us/step - loss: 0.5709 - acc: 0.7628 - val_loss: 0.5213 - val_acc: 0.7854
Epoch 143/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 36us/step - loss: 0.5678 - acc: 0.7640 - val_loss: 0.5246 - val_acc: 0.7832
Epoch 144/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 36us/step - loss: 0.5678 - acc: 0.7613 - val_loss: 0.5193 - val_acc: 0.7835
Epoch 145/500
Learning rate:  0.0001
47520/47520 [=========

47520/47520 [==============================] - 2s 36us/step - loss: 0.5585 - acc: 0.7665 - val_loss: 0.5136 - val_acc: 0.7842
Epoch 190/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 36us/step - loss: 0.5587 - acc: 0.7657 - val_loss: 0.5121 - val_acc: 0.7860
Epoch 191/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 37us/step - loss: 0.5589 - acc: 0.7680 - val_loss: 0.5104 - val_acc: 0.7860
Epoch 192/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 36us/step - loss: 0.5568 - acc: 0.7695 - val_loss: 0.5092 - val_acc: 0.7859
Epoch 193/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 36us/step - loss: 0.5577 - acc: 0.7661 - val_loss: 0.5118 - val_acc: 0.7857
Epoch 194/500
Learning rate:  0.0001
47520/47520 [==============================] - 2s 40us/step - loss: 0.5575 - acc: 0.7673 - val_loss: 0.5099 - val_acc: 0.7884
Epoch 195/500
Learning rate:  0.0001
47520/47520 [=========

...

Epoch 290/500
Learning rate:  1e-05
47520/47520 [==============================] - 2s 35us/step - loss: 0.5496 - acc: 0.7710 - val_loss: 0.5049 - val_acc: 0.7884
Epoch 291/500
Learning rate:  1e-05
47520/47520 [==============================] - 2s 35us/step - loss: 0.5502 - acc: 0.7698 - val_loss: 0.5052 - val_acc: 0.7874
Epoch 292/500
Learning rate:  1e-05
47520/47520 [==============================] - 2s 35us/step - loss: 0.5510 - acc: 0.7707 - val_loss: 0.5052 - val_acc: 0.7874
Epoch 293/500
Learning rate:  1e-05
47520/47520 [==============================] - 2s 35us/step - loss: 0.5502 - acc: 0.7710 - val_loss: 0.5050 - val_acc: 0.7880
Epoch 294/500
Learning rate:  1e-05
47520/47520 [==============================] - 2s 35us/step - loss: 0.5516 - acc: 0.7699 - val_loss: 0.5051 - val_acc: 0.7882
Epoch 295/500
Learning rate:  1e-05
47520/47520 [==============================] - 2s 35us/step - loss: 0.5503 - acc: 0.7715 - val_loss: 0.5052 - val_acc: 0.7892
Epoch 296/500
Learning rate:

47520/47520 [==============================] - 2s 35us/step - loss: 0.5499 - acc: 0.7709 - val_loss: 0.5049 - val_acc: 0.7884
Epoch 341/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5517 - acc: 0.7700 - val_loss: 0.5049 - val_acc: 0.7882
Epoch 342/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5514 - acc: 0.7678 - val_loss: 0.5049 - val_acc: 0.7884
Epoch 343/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5503 - acc: 0.7700 - val_loss: 0.5048 - val_acc: 0.7882
Epoch 344/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5493 - acc: 0.7709 - val_loss: 0.5048 - val_acc: 0.7886
Epoch 345/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5485 - acc: 0.7711 - val_loss: 0.5048 - val_acc: 0.7886
Epoch 346/500
Learning rate:  1e-06
47520/47520 [===============

Epoch 391/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5513 - acc: 0.7692 - val_loss: 0.5046 - val_acc: 0.7884
Epoch 392/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5492 - acc: 0.7710 - val_loss: 0.5046 - val_acc: 0.7886
Epoch 393/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5498 - acc: 0.7716 - val_loss: 0.5046 - val_acc: 0.7884
Epoch 394/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5523 - acc: 0.7714 - val_loss: 0.5046 - val_acc: 0.7884
Epoch 395/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 36us/step - loss: 0.5500 - acc: 0.7683 - val_loss: 0.5046 - val_acc: 0.7884
Epoch 396/500
Learning rate:  1e-06
47520/47520 [==============================] - 2s 35us/step - loss: 0.5508 - acc: 0.7690 - val_loss: 0.5046 - val_acc: 0.7884
Epoch 397/500
Learning rate:

47520/47520 [==============================] - 2s 35us/step - loss: 0.5489 - acc: 0.7691 - val_loss: 0.5046 - val_acc: 0.7889
Epoch 442/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5480 - acc: 0.7706 - val_loss: 0.5046 - val_acc: 0.7889
Epoch 443/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5484 - acc: 0.7713 - val_loss: 0.5046 - val_acc: 0.7887
Epoch 444/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5492 - acc: 0.7711 - val_loss: 0.5046 - val_acc: 0.7889
Epoch 445/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5526 - acc: 0.7673 - val_loss: 0.5046 - val_acc: 0.7889
Epoch 446/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5507 - acc: 0.7679 - val_loss: 0.5046 - val_acc: 0.7889
Epoch 447/500
Learning rate:  5e-07
47520/47520 [===============

Epoch 492/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5489 - acc: 0.7703 - val_loss: 0.5045 - val_acc: 0.7889
Epoch 493/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 35us/step - loss: 0.5512 - acc: 0.7681 - val_loss: 0.5045 - val_acc: 0.7889
Epoch 494/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 36us/step - loss: 0.5488 - acc: 0.7708 - val_loss: 0.5045 - val_acc: 0.7887
Epoch 495/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 36us/step - loss: 0.5510 - acc: 0.7703 - val_loss: 0.5045 - val_acc: 0.7889
Epoch 496/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 36us/step - loss: 0.5494 - acc: 0.7704 - val_loss: 0.5045 - val_acc: 0.7889
Epoch 497/500
Learning rate:  5e-07
47520/47520 [==============================] - 2s 36us/step - loss: 0.5519 - acc: 0.7694 - val_loss: 0.5045 - val_acc: 0.7889
Epoch 498/500
Learning rate:

In [11]:
score = model.evaluate(test, test_target_cat, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

5940/5940 [==============================] - 0s 7us/step

Test accuracy: 79.9%


In [12]:
test_df_result = model.predict_classes(test_df)
for i in range(test_df_result.shape[-1]):
    print(test_df_id[i],labels[test_df_result[i]])

50785 non functional
51630 functional
17168 functional
45559 non functional
49871 functional
52449 functional
24806 non functional
28965 non functional
36301 non functional
54122 functional
419 functional
45750 non functional
653 non functional
14017 functional
44607 functional
40228 functional
27714 functional
28785 functional
28330 functional
18532 non functional
69961 functional
55083 functional
8691 non functional
30331 non functional
70970 functional
61136 functional
28799 non functional
46825 non functional
44718 functional needs repair
37350 functional
65738 functional
3271 non functional
42658 non functional
67523 functional
37034 functional
42432 functional
27420 functional
707 functional
48489 non functional
51008 non functional
72562 functional
10275 functional
53540 non functional
43291 non functional
8813 functional
62343 non functional
1649 functional
8822 functional
10352 functional
50878 functional
15266 functional
51956 functional
44099 functional
59 functional
51359 n

60179 functional
46608 functional
7912 non functional
40483 functional
28335 non functional
64791 functional
970 functional
54743 functional
70993 functional
46287 functional
27169 non functional
29471 non functional
71476 functional
45420 functional
71980 functional
72862 functional
6168 functional
73489 non functional
67595 functional
37418 functional
23319 functional
379 functional
61286 functional
15933 functional
55858 functional
60144 non functional
4352 non functional
9905 non functional
27743 functional
35256 functional
35537 functional
25120 functional
43468 functional
53035 functional
69597 non functional
1440 functional
54701 functional
50701 functional
26306 non functional
5101 non functional
50562 functional
21381 functional
61186 functional needs repair
10726 functional
38114 functional
52810 functional
20389 non functional
43615 functional needs repair
57068 functional
49109 functional
11086 non functional
73675 functional
12727 functional
48628 functional
34433 function

3890 functional
51633 non functional
54844 functional
33149 functional
40421 non functional
27362 functional
66893 non functional
31148 functional
3362 non functional
51273 functional
2769 functional
31711 functional
31865 functional
30803 functional
8755 non functional
48751 functional needs repair
45069 functional
35483 non functional
10863 functional
46944 functional
17175 functional
63331 functional
33245 functional
47058 functional
31091 functional
21102 functional
15897 functional
60918 functional
15964 non functional
63592 functional
48162 non functional
55336 functional
23352 functional
6273 non functional
21356 functional
5093 functional
13518 functional
63302 non functional
67702 non functional
42060 functional
34084 non functional
39302 functional
43093 functional
6677 functional
60475 functional
58084 functional
27102 functional
49393 non functional
32029 functional
2534 functional
29386 functional
72884 non functional
49553 functional
37640 functional
52508 functional
3422

12594 functional
17034 functional
60913 functional
42050 functional
6973 non functional
46869 functional
17457 functional
54619 functional
52675 functional
27072 non functional
7849 non functional
62488 functional
70763 functional
13032 functional
54851 functional
67968 non functional
21690 functional
56415 non functional
31352 non functional
54253 functional
23650 non functional
62683 non functional
35693 functional
25763 functional
72950 non functional
37896 functional
54366 non functional
39279 non functional
43971 functional
2467 functional
72516 functional
17266 non functional
41746 non functional
9315 functional
9802 functional
73506 non functional
5285 non functional
16827 functional
58094 non functional
44719 functional
18637 functional
18424 functional
62010 functional
8026 functional needs repair
37848 functional
13326 functional
7242 functional
46232 non functional
19179 functional
14255 functional
32278 functional
10351 non functional
67825 non functional
21183 non function

14842 non functional
67294 non functional
61106 non functional
61693 functional
18196 functional
27990 functional
17475 functional
19280 non functional
43449 non functional
5510 functional
9456 non functional
20064 functional
5036 functional
23634 functional
32625 non functional
232 non functional
20621 functional
73340 functional needs repair
41313 non functional
19027 functional
59172 non functional
13918 non functional
32798 functional
72071 non functional
59642 functional
25338 functional
7263 functional
5181 functional
36311 non functional
41021 functional
25507 functional
13061 functional
62816 functional
34508 functional
72432 non functional
6206 functional
38985 functional
1344 functional
56627 non functional
29762 functional
40935 functional
25881 functional
66780 functional
70833 non functional
59072 functional
48905 non functional
37413 non functional
55075 non functional
43785 functional
6186 non functional
20044 functional
45487 non functional
62706 functional
67613 functi

18328 functional
14412 functional
11893 non functional
72023 functional
66343 functional
45382 functional
23285 non functional
57381 functional
8369 non functional
66506 non functional
23899 functional
27444 functional
30988 non functional
61062 functional
50980 functional
31529 functional
44539 functional
47819 functional
34276 non functional
31288 functional
51303 functional
73437 functional
26960 functional
68632 functional
66792 non functional
39714 functional
52431 non functional
72339 functional
46060 functional
20038 functional
4705 non functional
67912 non functional
11300 functional
72175 functional
42554 functional
58123 non functional
26228 functional
143 functional
63371 functional
64300 functional
31506 non functional
54092 functional
18340 functional
24853 non functional
40667 functional
60777 functional
51544 functional
23956 functional
51949 non functional
15018 functional
23297 functional
25254 non functional
17197 functional
34104 non functional
48087 non functional
5

59337 functional
39037 functional
671 functional
41286 non functional
37572 functional
67085 functional
60724 non functional
4426 non functional
33519 functional
68693 functional
27411 functional
16059 non functional
10508 functional
51091 functional
58161 non functional
2009 functional
65306 functional
45129 functional
70886 functional
21079 non functional
14423 functional
29260 functional
33415 functional
22153 functional
55873 functional
56274 functional
973 non functional
2449 non functional
50726 functional
56206 non functional
11729 functional
13638 functional
58446 functional
9961 functional
3172 functional
19777 functional
5681 functional
47297 functional
68606 non functional
58589 functional
70214 functional
61042 non functional
11747 functional
72704 functional
19528 functional
25812 non functional
9042 functional
52702 functional
60287 functional
67148 functional
54592 functional
64936 non functional
17514 functional
31598 functional
12279 functional
64840 functional
4094 fu

In [13]:
test_df_pred = pd.DataFrame(labels[test_df_result])
test_df_pred.columns = ['status_group']
test_df_id = pd.DataFrame(test_df_id)
test_df_pred = test_df_id.join(test_df_pred)

In [14]:
pd.DataFrame(test_df_pred).to_csv("submission.csv", index=False)